In [1]:
import os
import autogen
from autogen.cache import Cache

In [2]:
import sys
import os
# Add the parent directory to the system path
parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_directory not in sys.path:
    sys.path.insert(0, parent_directory)

In [3]:
print(sys.path)

['/Users/yilin/opt/anaconda3/envs/finrobotragup/lib/python3.10/site-packages/ray/thirdparty_files', '/Users/yilin/opt/anaconda3/envs/finrobotragup/lib/python310.zip', '/Users/yilin/opt/anaconda3/envs/finrobotragup/lib/python3.10', '/Users/yilin/opt/anaconda3/envs/finrobotragup/lib/python3.10/lib-dynload', '', '/Users/yilin/opt/anaconda3/envs/finrobotragup/lib/python3.10/site-packages', '/Users/yilin/FinRobot']


In [4]:
!pip install langchain-chroma -U -q
!pip install sentence-transformers -q

In [3]:
from finrobot.data_source.earnings_calls_src.main_earningsData import get_earnings_all_docs
from finrobot.data_source.earnings_calls_src import earningsData
from finrobot.data_source.filings_src import sec_main as unstructured_sec_main
from finrobot.data_source.earnings_calls_src.main_earningsData import get_earnings_all_docs
from finrobot.data_source.earnings_calls_src import earningsData
#from finrobot.data_source.finance_data import get_data
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [4]:
from finrobot.functional.ragqueryupup import rag_database_earnings_call
#from finrobot.functional.ragqueryupup import rag_database_unstructured_sec
from finrobot.functional.ragqueryupup import rag_database_sec

In [5]:
#ticker = 'GOOG'
ticker = 'NVDA'
year = '2023'
filing_types = ['10-K','10-Q']
include_amends = True

In [6]:
query_database_earnings_call, earnings_call_quarter_vals, quarter_speaker_dict = rag_database_earnings_call(ticker = ticker, year = year)

Earnings Call Q1
Earnings Call Q2
Earnings Call Q3
Earnings Call Q4


In [7]:
global FROM_MARKDOWN
FROM_MARKDOWN = False

In [8]:
query_database_sec, sec_form_names = rag_database_sec(ticker = ticker, year = year,FROM_MARKDOWN = FROM_MARKDOWN)

Started Scraping
Scraped
Started Extracting
Extracted


In [9]:
sec_form_system_msg = ""
#llm_config = {"model":"gpt-4-turbo"}
for sec_form in sec_form_names:
    if sec_form == "10-K":
        sec_form_system_msg+= "10-K for yearly data, "
    elif "10-Q" in sec_form:
        quarter = sec_form[-1]
        sec_form_system_msg+= f"{sec_form} for Q{quarter} data, "
sec_form_system_msg = sec_form_system_msg[:-2]

earnings_call_system_message = ", ".join(earnings_call_quarter_vals)

system_msg = f"""You are a helpful financial assistant and your task is to select the sec_filings or earnings_call or financial_books to best answer the question.
You can use query_database_sec(question,sec_form) by passing question, relevant sec_form names like {sec_form_system_msg}
or you can use query_database_earnings_call(question,quarter) by passing question and relevant quarter names with possible values {earnings_call_system_message}
or you can use query_database_books(question) to get relevant documents from financial textbooks about valuation and investing philosophies. When you are ready to end the coversation, reply TERMINATE"""

In [10]:
from autogen import ConversableAgent
import os


config_list = autogen.config_list_from_json(
    "../OAI_CONFIG_LIST1",
    filter_dict={
        "model": ["gpt-4-0125-preview"],
    },
)
llm_config = {
    "config_list": config_list,
    "timeout": 120,
    # "temperature": 0 # for debug convenience
    "temperature": 0.5,
}
user_proxy = ConversableAgent(
    name = "Planner Admin",
    system_message=system_msg,
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
)
tool_proxy = ConversableAgent(
  name="Tool Proxy",
  system_message="Analyze the response from user proxy and decide whether the suggested database is suitable "
  ". Answer in simple yes or no",
  llm_config=False,
  # is_termination_msg=lambda msg: "exit" in msg.get("content",""),
  default_auto_reply="Please select the right database.",
  human_input_mode="ALWAYS",
  )

tools_dict = {
        "sec":[query_database_sec,"Tool to query SEC filings database"],
        "earnings_call": [query_database_earnings_call, "Tool to query earnings call transcripts database"],
    }

In [11]:
from autogen import register_function

for tool_name,tool in tools_dict.items():
  register_function(
      tool[0],
      caller=user_proxy,
      executor=tool_proxy,
      name = tool[0].__name__,
      description=tool[1]
  )

In [13]:
input_text = "What is the strategy of Nvidia for artificial intelligence?"
#input_text = "What is the strategy of Google for artificial intelligence?"
chat_result = user_proxy.initiate_chat(
        recipient=tool_proxy,
        message=input_text,
        max_turns=5
    )

Planner Admin (to Tool Proxy):

What is the strategy of Nvidia for artificial intelligence?

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

***** Suggested tool call (call_Vpand1auVseecRK4RhOqSO1F): query_database_unstructured_sec *****
Arguments: 
{"question": "What is the strategy of Nvidia for artificial intelligence?", "sec_form_name": "10-K"}
************************************************************************************************
***** Suggested tool call (call_lvLhiFM4RggktJWRdhuR52le): query_database_earnings_call *****
Arguments: 
{"question": "What is the strategy of Nvidia for artificial intelligence?", "quarter": "Q3"}
*********************************************************************************************

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION query_database_unstructured_sec...


...

Tool Proxy (to Planner Admin):

Tool Proxy (to Planner Admin):

***** Response from calling tool (call_Vpand1auVseecRK4RhOqSO1F) *****
FORM_SUMMARY: any of these key success criteria, our financial results may be harmed. We will offer enterprise customers NVIDIA AI cloud services directly and through our network of partners. Examples of these services include NVIDIA DGX Cloud, which is cloud-based infrastructure and software for training AI models, and customizable pretrained AI models. NVIDIA has partnered with leading cloud service providers to host these services in their data centers, and we entered into multi-year cloud service agreements in the second half of fiscal year 2023 to support these offerings and our research and development activities. NVIDIA AI cloud services may not be successful and will take time, resources and investment. We also offer or plan to offer standalone software solutions for AI including NVIDIA AI Enterprise, NVIDIA Omniverse, NVIDIA DRIVE for automotiv

Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

TERMINATE

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.


In [21]:
input_text = "What was forward estimates of Google for the year 2023?"
chat_result = user_proxy.initiate_chat(
        recipient=tool_proxy,
        message=input_text,
        max_turns=10
    )

Planner Admin (to Tool Proxy):

What was forward estimates of Google for the year 2023?

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

***** Suggested tool call (call_L8egUxMdh0EVYQLMDPdEFFg6): query_database_unstructured_sec *****
Arguments: 
{"question":"What was forward estimates of Google for the year 2023?","sec_form_name":"10-K"}
************************************************************************************************

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION query_database_unstructured_sec...


...


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

The right database to query for forward estimates of Google for the year 2023 would be the SEC filings, specifically the 10-K form, which I have already queried.

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

TERMINATE

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.


In [13]:
input_text = "What are the risk factors that Nvidia faced this year?"
chat_result = user_proxy.initiate_chat(
        recipient=tool_proxy,
        message=input_text,
        max_turns=10
    )

Planner Admin (to Tool Proxy):

What are the risk factors that Nvidia faced this year?

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

***** Suggested tool call (call_ZOWSOgPFUQTIuX4aHLbMTFST): query_database_unstructured_sec *****
Arguments: 
{"question":"What are the risk factors that Nvidia faced this year?","sec_form_name":"10-K"}
************************************************************************************************

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION query_database_unstructured_sec...


...

Tool Proxy (to Planner Admin):

Tool Proxy (to Planner Admin):

***** Response from calling tool (call_ZOWSOgPFUQTIuX4aHLbMTFST) *****
FORM_SUMMARY: In evaluating NVIDIA, the following risk factors should be considered in addition to the other information in this Annual Report on Form 10-K. Purchasing or owning NVIDIA common stock involves investment risks including, but not limited to, the risks described below. Any one of the following risks could harm our business, financial condition, results of operations or reputation, which could cause our stock price to decline, and you may lose all or a part of your investment. Additional risks, trends and uncertainties not presently known to us or that we currently believe are immaterial may also harm our business, financial condition, results of operations or reputation. Failure to meet the evolving needs of our industry and markets may adversely impact our financial results. Competition in our current and target markets could cause us to lo

Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
Tool Proxy (to Planner Admin):

Please select the right database.

--------------------------------------------------------------------------------
Planner Admin (to Tool Proxy):

TERMINATE

--------------------------------------------------------------------------------


Provide feedback to Planner Admin. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.
